# Basketball-Reference Team Data analysis

In [1]:
# special IPython command to prepare the notebook for matplotlib
from collections import defaultdict

import pandas as pd
import matplotlib.pyplot as plt
import requests
import re
import string
from pattern import web

### Read Player Seaaon CSV

In [7]:
df_player_season = pd.read_csv('e:\\Season_Totals.csv', header=0)

df_player_season.info()
#df_player_season.describe()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3625 entries, 0 to 3624
Data columns (total 33 columns):
Season         3625 non-null object
Age            3625 non-null int64
Tm             3625 non-null object
Lg             3625 non-null object
Pos            3625 non-null object
G              3625 non-null int64
GS             3625 non-null int64
MP             3625 non-null int64
FG             3625 non-null int64
FGA            3625 non-null int64
FG_Per         3617 non-null float64
three_P        3625 non-null int64
three_PA       3625 non-null int64
three_P_Per    3154 non-null float64
two_P          3625 non-null int64
two_PA         3625 non-null int64
two_P_Per      3613 non-null float64
eFG_Per        3617 non-null float64
FT             3625 non-null int64
FTA            3625 non-null int64
FT_Per         3567 non-null float64
ORB            3625 non-null int64
DRB            3625 non-null int64
TRB            3625 non-null int64
AST            3625 non-null int64
STL 

### Read Team CSV

In [23]:
df_team = pd.read_csv('e:\\nba_team.csv', header=0)

df_team.info()
#df_team.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180 entries, 0 to 179
Data columns (total 26 columns):
tm_Tm           180 non-null object
tm_G            180 non-null int64
tm_MP           180 non-null int64
tm_FG           180 non-null int64
tm_FGA          180 non-null int64
tm_FG_Per       180 non-null float64
tm_three_P      180 non-null int64
tm_three_PA     180 non-null int64
tm_three_P.1    180 non-null float64
tm_twoP         180 non-null int64
tm_twoPA        180 non-null int64
tm_twoP.1       180 non-null float64
tm_FT           180 non-null int64
tm_FTA          180 non-null int64
tm_FT_Per       180 non-null float64
tm_ORB          180 non-null int64
tm_DRB          180 non-null int64
tm_TRB          180 non-null int64
tm_AST          180 non-null int64
tm_STL          180 non-null int64
tm_BLK          180 non-null int64
tm_TOV          180 non-null int64
tm_PF           180 non-null int64
tm_PTS          180 non-null int64
tm_PTSG         180 non-null float64
Season   

### Data split

In [9]:
df_player_season['Age'].describe()

count    3625.000000
mean       25.461517
std         3.702994
min        18.000000
25%        23.000000
50%        25.000000
75%        28.000000
max        39.000000
Name: Age, dtype: float64

In [41]:
#df_MIA=df_team[(df_team['Season'] == "2013-14") & (df_team['tm_Tm'] == "MIA")]
df_team_2015 = df_team[(df_team['Season'] == "2014-2015")]
df_team_2015
#df_2015.describe().mean()

#df_CLE = df_team[(df_team['Season'] == "2014-2015") & (df_team['tm_Tm'] == "CLE")]

,tm_Tm,tm_G,tm_MP,tm_FG,tm_FGA,tm_FG_Per,tm_three_P,tm_three_PA,tm_three_P.1,tm_twoP,...,tm_DRB,tm_TRB,tm_AST,tm_STL,tm_BLK,tm_TOV,tm_PF,tm_PTS,tm_PTSG,Season
150,MIL,82,19930,3083,6722,0.459,545,1500,0.363,2538,...,2574,3450,1932,789,403,1373,1814,8023,97.8,2014-2015
151,GSW,82,19730,3410,7137,0.478,883,2217,0.398,2527,...,2814,3667,2248,762,496,1185,1628,9016,110.0,2014-2015
152,MIN,82,19805,2986,6820,0.438,406,1223,0.332,2580,...,2406,3355,1771,668,327,1231,1571,8016,97.8,2014-2015
153,TOR,82,19855,3108,6829,0.455,726,2060,0.352,2382,...,2526,3407,1701,615,357,1057,1712,8527,104.0,2014-2015
154,ATL,82,19730,3121,6699,0.466,818,2152,0.380,2303,...,2611,3326,2111,744,380,1167,1457,8409,102.5,2014-2015
155,BOS,82,19880,3193,7211,0.443,660,2021,0.327,2533,...,2685,3595,2009,674,294,1133,1738,8312,101.4,2014-2015
156,DET,82,19830,3041,7038,0.432,703,2043,0.344,2338,...,2630,3681,1771,623,383,1099,1559,8077,98.5,2014-2015
157,NYK,82,19855,2882,6726,0.428,560,1614,0.347,2322,...,2443,3310,1746,575,382,1206,1768,7535,91.9,2014-2015
158,DEN,82,19880,3099,7158,0.433,660,2032,0.325,2439,...,2653,3665,1788,641,367,1166,1882,8320,101.5,2014-2015
159,SAC,82,19855,3010,6617,0.455,461,1350,0.341,2549,...,2728,3623,1667,550,324,1333,1696,8310,101.3,2014-2015


In [49]:
df_player=df_player_season[['Tm','Season']].head(1)
df_player

,Tm,Season
0,OKC,2010-11
